In [35]:
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser
import pandas as pd

from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)



sentences = Text8Corpus(datapath('/home/tiago_duque/Área de Trabalho/Dissertação/textO/data/Raw/500perguntasgadoleite.txt'))
phrases = Phrases(sentences, min_count=2, threshold=1)
termlist = pd.read_csv('/home/tiago_duque/Área de Trabalho/Dissertação/textO/data/TreatedTerms/tfidfterms.csv')

sent = termlist['0'].tolist()
termlist = [term for term in termlist if len(term) >2]
#print(sent)

bigram = Phraser(phrases)
#sent = [u'gado', u'leite', u'brachiaria', u'decumbens']
for sent in sentences:
    tokens_ = bigram[sent]
    for token in tokens_:
        if '_' in token:
            words = token.split("_")
            w0l = lemmatizer(words[0],'NOUN')[0]
            w1l = lemmatizer(words[1],'ADJ')[0]
            if (words[0] in termlist or w0l in termlist) and (words[1] in termlist or w1l in termlist):
                print(token)

In [4]:
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk import Text
from nltk import word_tokenize

stwords = set(stopwords.words('portuguese'))

root = 'data/Raw/quinhentasCorpus/'
reader = PlaintextCorpusReader(root,'.*','.txt')
raw = reader.raw()
tokens = word_tokenize(raw)
text = Text(tokens)
text.collocations()
termlist = pd.read_csv('/home/tiago_duque/Área de Trabalho/Dissertação/textO/data/TreatedTerms/tfidfterms.csv')


deve ser; devem ser; pode ser; dos animais; sal mineral; Além disso;
podem ser; matéria seca; inseminação artificial; acordo com; azevém
anual; condição corporal; glândula mamária; ser feita; melhoramento
genético; valor nutritivo; raças europeias; Brachiaria decumbens;
células somáticas; médico veterinário


In [5]:
from nltk.collocations import *
import pandas as pd
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
tfinder = TrigramCollocationFinder.from_words(tokens)
finder = BigramCollocationFinder.from_words(tokens)
ignored_words = nltk.corpus.stopwords.words('portuguese')
finder.apply_word_filter(lambda w: len(w) < 4 or w.lower() in ignored_words and w.lower() in termlist)
finder.apply_freq_filter(5)
tfinder.apply_word_filter(lambda w: len(w) < 4 or w.lower() in ignored_words and w.lower() in termlist)
tfinder.apply_freq_filter(10)
bilist = finder.nbest(bigram_measures.pmi, 200)
trilist = tfinder.nbest(trigram_measures.pmi, 200)
trilist = [" ".join(t) for t in trilist]
bilist = [" ".join(b) for b in bilist]
## Eliminates duplicate
bilist = [b for b in bilist if all(b not in t for t in trilist)]

tdf1 = pd.DataFrame(bilist)
tdf2 = pd.DataFrame(trilist)

tdf1.to_csv("data/TreatedTerms/bigrams.csv")
tdf2.to_csv("data/TreatedTerms/trigrams.csv")


In [33]:
from nltk.collocations import *
import pandas as pd
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk import Text
from nltk import word_tokenize
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

quest = pd.read_csv('data/Raw/quinhentas_perguntas.csv')
ignored_words = nltk.corpus.stopwords.words('portuguese')

words = set()
for idx, row in quest.iterrows():
    words = words.union(set([w for w in word_tokenize(row[1]) if w not in ignored_words]))
fulltext = ""
termlist = pd.read_csv('/home/tiago_duque/Área de Trabalho/Dissertação/textO/data/TreatedTerms/tfidfterms.csv')
termlist = list(termlist['0'])
toremove = set()
for word in words:
    if word in termlist or lemmatizer(word, "NOUN")[0] in termlist or lemmatizer(word,"VERB")[0] in termlist:
        toremove.add(word)
for word in toremove:
    words.remove(word)
for idx, row in quest.iterrows():
    fulltext+=row[1]
tokens = word_tokenize(fulltext)
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
tfinder = TrigramCollocationFinder.from_words(tokens)
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_word_filter(lambda w: len(w) < 4 or w.lower() in ignored_words and w.lower() in words)
finder.apply_freq_filter(3)
tfinder.apply_word_filter(lambda w: len(w) < 4 or w.lower() in ignored_words and w.lower() in words)
tfinder.apply_freq_filter(5)
bilist = finder.nbest(bigram_measures.pmi, 200)
trilist = tfinder.nbest(trigram_measures.pmi, 200)
trilist = [" ".join(t) for t in trilist]
bilist = [" ".join(b) for b in bilist]
## Eliminates duplicate
bilist = [b for b in bilist if all(b not in t for t in trilist)]
words = [w for w in words if len(w) > 2]
words = set([lemmatizer(w, "NOUN")[0] for w in words])
words = set([lemmatizer(w, "VERB")[0] for w in words])
df = pd.DataFrame({"term":list(words)})
df.to_csv('data/TreatedTerms/wordsFromQuestions.csv')

